<h1 style="color:#BF66F2 ">  Recurrent Neural Networks in PyTorch 2 </h1>
<div style="margin-top: -30px;">
<h4> Predict names given the nationality using LSTMs. </h4> 
</div>
<div style="margin-top: -18px;">
<span style="display: inline-block;">
    <h3 style="color: lightblue; display: inline;">Keywords:</h3> NLLLoss + autograd + colab upload + zipfile + SOS token
</div>

In [17]:
import sys
import glob
import time
import math
import random

import string
import decimal
import zipfile
import unicodedata

import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.autograd
import torch.nn as nn
import torch.optim as optim

In [18]:
#%%script echo Skipping. Uncomment if not on Colab
from google.colab import files

In [19]:
#%%script echo Skipping. Uncomment if not on Colab
uploaded_zip = files.upload()

In [20]:
#%%script echo Skipping. Uncomment if not on Colab
# Extract the uploaded zip archive
with zipfile.ZipFile('names_nationality.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [36]:
!ls /content/names_nationality

arabic.txt   english.txt  irish.txt	polish.txt	spanish.txt
chinese.txt  french.txt   italian.txt	portoguese.txt	vietnamese.txt
czech.txt    german.txt   japanese.txt	russian.txt
dutch.txt    greek.txt	  korean.txt	scottish.txt


In [37]:
def find_files_glob(path): 
    return glob.glob(path)

print("Files found:", find_files_glob('/content/names_nationality/*.txt'))

Files found: ['/content/names_nationality/english.txt', '/content/names_nationality/french.txt', '/content/names_nationality/dutch.txt', '/content/names_nationality/czech.txt', '/content/names_nationality/korean.txt', '/content/names_nationality/italian.txt', '/content/names_nationality/chinese.txt', '/content/names_nationality/spanish.txt', '/content/names_nationality/polish.txt', '/content/names_nationality/arabic.txt', '/content/names_nationality/russian.txt', '/content/names_nationality/portoguese.txt', '/content/names_nationality/scottish.txt', '/content/names_nationality/greek.txt', '/content/names_nationality/japanese.txt', '/content/names_nationality/vietnamese.txt', '/content/names_nationality/irish.txt', '/content/names_nationality/german.txt']


In [ ]:
def from_letter_to_index(letter):
    return all_letters.index(letter)

all_letters = list(string.ascii_letters + " .,;'")
all_letters += ['<EOS>', '<SOS>']
n_letters = len(all_letters)

def from_unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

def read_all_lines(filename):
    """ Read a file and split into lines """
    lines = open(filename).read().strip().split('\n')
    return [from_unicode_to_ascii(line) for line in lines]

In [ ]:
all_categories, category_lines = [], {}

In [39]:
""" Create all categories.
N.B.
Use this is not on Colab:
=> 'for filename in findFiles('./dataset/names/names_nationality/*.txt'):'
"""

for filename in find_files_glob('/content/names_nationality/*.txt'):
    print("ciao")
    category = filename.split('/')[-1].split('.')[0]
    print("category", category)
    all_categories.append(category)
    lines = read_all_lines(filename)
    category_lines[category] = lines
    print("all_categories")
    print(all_categories)

n_categories = len(all_categories)

ciao
category english
all_categories
['english']
ciao
category french
all_categories
['english', 'french']
ciao
category dutch
all_categories
['english', 'french', 'dutch']
ciao
category czech
all_categories
['english', 'french', 'dutch', 'czech']
ciao
category korean
all_categories
['english', 'french', 'dutch', 'czech', 'korean']
ciao
category italian
all_categories
['english', 'french', 'dutch', 'czech', 'korean', 'italian']
ciao
category chinese
all_categories
['english', 'french', 'dutch', 'czech', 'korean', 'italian', 'chinese']
ciao
category spanish
all_categories
['english', 'french', 'dutch', 'czech', 'korean', 'italian', 'chinese', 'spanish']
ciao
category polish
all_categories
['english', 'french', 'dutch', 'czech', 'korean', 'italian', 'chinese', 'spanish', 'polish']
ciao
category arabic
all_categories
['english', 'french', 'dutch', 'czech', 'korean', 'italian', 'chinese', 'spanish', 'polish', 'arabic']
ciao
category russian
all_categories
['english', 'french', 'dutch', 'cz

In [40]:
def get_name_tensor(name):
    """ Convert a given name string into a LongTensor,\\
    where each character is an index determined by the "from_letter_to_index" function.
    """
    res = []
    for car in name:
        res.append(from_letter_to_index(car))
    return torch.LongTensor(res)

def get_category_of_tensor(category):
    """ Convert a given category string into a LongTensor. """ 
    li = all_categories.index(category)
    return torch.LongTensor([li])

def get_input_of_tensor(line):
    """ Convert a given line string into a LongTensor,\\
        where each character is an index determined by the "from_letter_to_index" function.\\
        The tensor starts with a '<SOS>' token.
    """    
    res = []
    res.append(from_letter_to_index('<SOS>'))
    for li in range(len(line)):
        letter = line[li]
        res.append(from_letter_to_index(letter))
    tensor = torch.LongTensor(res)
    return tensor

In [41]:
DIM_EMBEDDING_CAR = 110
DIM_EMBEDDING_CAT = 18

class RNN(nn.Module):
    """ Recurrent Neural Network (RNN) model (Long Short-Term Memory). 
        
        Parameters:        
            - Size of the input feature vector [int]
            - Number of hidden units in the LSTM layer [int]
            - Size of the output feature vector [int]
        
        Attributes:
            - hidden_size: Number of hidden units in the LSTM layer [int]
            - embedding_car: Embedding layer for input feature vectors of cars [torch.nn.Embedding]
            - embedding_cat: 
                Embedding layer for categories [torch.nn.Embedding]
            - lstm: LSTM layer that takes embedded inputs and outputs hidden states with 'hidden_size' hidden units. [torch.nn.LSTM]
            - toProbNextLetter:  Linear layer that maps the output of the LSTM layer to the output size,\\
            acting as a classifier to predict the next letter in a sequence [torch.nn.Linear]
            - softmax: LogSoftmax layer that converts output of the previous 'toProbNextLetter' linear layer to log-probability values. [torch.nn.LogSoftmax]
        
        Notes:
            - Try always softmax for the final output with multiclass classification problems.
            - A dropout layer can be added to mitigate overfitting during training.
    """    
    
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding_car = nn.Embedding(input_size, DIM_EMBEDDING_CAR)
        self.embedding_cat = nn.Embedding(len(all_categories), DIM_EMBEDDING_CAT)
        self.lstm = nn.LSTM(DIM_EMBEDDING_CAT+DIM_EMBEDDING_CAR, hidden_size)
        self.toProbNextLetter = nn.Linear(hidden_size, output_size)
        #self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)


    def forward(self, category, input, hidden):
        car_embeds = self.embedding_car(input)
        cat_embeds = self.embedding_cat(category)
        cat_embeds = cat_embeds.view(1, -1).repeat(len(input), 1)

        combined_input = torch.cat((car_embeds, cat_embeds), 1)
        combined_input = combined_input.unsqueeze(1)

        lstm_out, hidden = self.lstm(combined_input, hidden)

        output = self.toProbNextLetter(lstm_out.view(-1, self.hidden_size))
        output = self.softmax(output)

        print(f"Input shape: {input.shape}")
        print(f"Car embeds shape: {car_embeds.shape}")
        print(f"Cat embeds shape: {cat_embeds.shape}")
        print(f"Cat embeds reshaped shape: {cat_embeds.shape}")
        print(f"Combined input shape: {combined_input.shape}")
        print(f"Combined input unsqueezed shape: {combined_input.shape}")
        print(f"LSTM output shape: {lstm_out.shape}")
        print(f"Output shape after linear: {output.shape}")
        print(f"Output shape after softmax: {output.shape}")
        return output, hidden

    def initHidden(self):
        return (Variable(torch.zeros(1, 1, self.hidden_size)), Variable(torch.zeros(1, 1, self.hidden_size)))

In [ ]:
EOS = n_letters - 2

def randomChoice(l):
    """ Get a random item from a list. """
    print(l)
    return l[random.randint(0, len(l) - 1)]

def randomTrainingPair():
    """ Get a random category and random line from that category. """
    category = randomChoice(all_categories)
    print("category ", category)
    line = randomChoice(category_lines[category])
    print("line ", line)
    return category, line

def get_long_target_tensor(line):
    """ Get a LongTensor of second letter to end (EOS) for target. """
    letter_indexes = [all_letters.index(line[li]) for li in range(0, len(line))]
    letter_indexes.append(EOS)
    return torch.LongTensor(letter_indexes)

def get_random_training_example():
    """ Get a random training example (with 3 tensors). """
    category, line = randomTrainingPair()
    category_tensor = Variable(get_category_of_tensor(category))
    input_line_tensor = Variable(get_input_of_tensor(line))
    print("input_line_tensor: " , input_line_tensor)
    target_line_tensor = Variable(get_long_target_tensor(line))
    return category_tensor, input_line_tensor, target_line_tensor

In [ ]:
rnn = RNN(n_letters, 128, n_letters)
criterion = nn.NLLLoss()
learning_rate = 0.0003
optimizer = optim.Adam(rnn.parameters(), learning_rate)

def train(category_tensor, input_line_tensor, target_line_tensor):
    """ Train the network.

    Parameters:
        - Category of the input sequence [torch.Tensor of shape (batch_size, input_size)]
        - Input sequence data [torch.Tensor of shape seq_len, batch_size, input_size]
        - Target sequence data[torch.Tensor with shape seq_len, batch_size, target_size]
        
    Returns:
        - Output tensor with model's predictions, typically of shape [torch.Tensor seq_len, batch_size, output_size]
        - Average loss per item in the sequence computed using the predefined criterion,\\ 
        calculated as the total loss divided by the sequence length.
    """

    optimizer.zero_grad()
    loss = 0
    hidden = rnn.initHidden()
    for i in range(input_line_tensor.size(0)):
        output, hidden = rnn(category_tensor, input_line_tensor, hidden)
        loss += criterion(output, target_line_tensor)
    loss.backward()
    optimizer.step()

    #return output, loss.data[0] / input_line_tensor.size(0)            #wrong!
    return output, loss.item() / input_line_tensor.size(0)


In [ ]:
def cal_time_since(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [ ]:
""" Hyperparameters """
n_iters = 100000
print_every = 500
save_every = 50000
all_losses = []
total_loss = 0
saves = 0

In [42]:
start = time.time()

for iter in range(1, n_iters + 1):
    category_tensor, input_line_tensor, target_line_tensor = get_random_training_example()
    print("#####")
    print(category_tensor)
    print(input_line_tensor)
    print(target_line_tensor)
    print("#####")
    output, loss = train(category_tensor, input_line_tensor, target_line_tensor)
    total_loss += loss

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (cal_time_since(start), iter, iter / n_iters * 100, total_loss))
        total_loss = 0

    if iter % save_every == 0:
        torch.save(rnn, 'char-rnn-generation{}.pt'.format(saves))
        saves+=1

Streaming output truncated to the last 5000 lines.
Output shape after softmax: torch.Size([9, 59])
Input shape: torch.Size([9])
Car embeds shape: torch.Size([9, 110])
Cat embeds shape: torch.Size([1, 18])
Cat embeds reshaped shape: torch.Size([9, 18])
Combined input shape: torch.Size([9, 128])
Combined input unsqueezed shape: torch.Size([9, 1, 128])
LSTM output shape: torch.Size([9, 1, 128])
Output shape after linear: torch.Size([9, 59])
Output shape after softmax: torch.Size([9, 59])
Input shape: torch.Size([9])
Car embeds shape: torch.Size([9, 110])
Cat embeds shape: torch.Size([1, 18])
Cat embeds reshaped shape: torch.Size([9, 18])
Combined input shape: torch.Size([9, 128])
Combined input unsqueezed shape: torch.Size([9, 1, 128])
LSTM output shape: torch.Size([9, 1, 128])
Output shape after linear: torch.Size([9, 59])
Output shape after softmax: torch.Size([9, 59])
Input shape: torch.Size([9])
Car embeds shape: torch.Size([9, 110])
Cat embeds shape: torch.Size([1, 18])
Cat embeds re

<h3 style="color:#BF66F2 ">  => Methods </h3>

In [ ]:
EOS = n_letters-2
SOS = n_letters-1
beam_size = 8

def get_softmax_index(top, j):
    return int(top[j]/n_letters)

def get_letter_index(top, j):
    return int(top[j]%n_letters)

def start_sampling(category):
    """ Beam Search Sampling for Name Generation given a category, exploiting an RNN model.
    
    Details:
        - It starts to generate names with a Start Of Sequence (SOS) token,\\
        then extends the generated sequence, maintaining a set number of the best candidates at each iteration. 
        - It stops when the max num of iterations is reached or an End Of Sequence (EOS) token is sampled.
    
    Parameters:
        - Category for which names are to be generated [str]

    Returns:
        - Generated names [list of str]
        - Scores for each generated name, normalized by name length [list of float]
    """

    # Convert the category to a PyTorch Variable tensor
    category_tensor = Variable(get_category_of_tensor(category))
    # Initialize the input as a Start Of Sequence (SOS) token, wrapped in a list inside a Variable tensor
    input = [Variable(torch.LongTensor([SOS]))]
    # Initialize hidden states for the RNN, multiplied by the beam size, and create a list of zeros for hidden probabilities
    hiddens = [rnn.initHidden()] * beam_size
    ##### Initilize 
    hprobs = [0] * beam_size
    names = [''] * beam_size
    scores, outs = [], []
    iters = 0

    while len(input) > 0:
        probs = []

        for j in range(0, len(input)):
            output, hiddens[j] = rnn(category_tensor, input[j], hiddens[j])
            output += hprobs[j]
            probs.append(output.data)

        if len(probs) == 1:
            probs = probs[0]
        else:
            # Concatenate probabilities. If there is more than one, else use the single set of probabilities
            probs = torch.cat(probs, 1)
        if iters != 0:
            how_many_samples = len(input)
        else:
            how_many_samples = beam_size

        topv, topi = probs.topk(how_many_samples)
        topv, topi = topv.numpy()[0], topi.numpy()[0]

        input = []
        old_names = list(names)
        old_hiddens = list(hiddens)
        old_hprobs = list(hprobs)
        hiddens, names, hprobs = [], [], []

        for j in range(how_many_samples):
            if get_letter_index(topi, j) != EOS:
                #### Update the hidden states, names, and probabilities, if the letter index is not an 'End Of Sequence' token
                hiddens.append(old_hiddens[get_softmax_index(topi, j)])
                names.append(old_names[get_softmax_index(topi, j)] + all_letters[get_letter_index(topi, j)])
                hprobs.append(float(topv[j]))
                input.append(Variable(torch.LongTensor([get_letter_index(topi, j)])))
            else:
                #### Append the name and score (normalized by length) to the outputs and scores lists, if an 'End Of Sequence' is found
                candidate = old_names[get_softmax_index(topi, j)]
                score = old_hprobs[get_softmax_index(topi, j)] / len(candidate)
                outs.append(candidate)
                scores.append(score)
        iters+=1

    return outs, scores

<h2 style="color:#BF66F2 ">  Main </h2>

In [44]:
for cat in all_categories:
    outs, scores = start_sampling(cat)
    print('====== {}'.format(cat))
    for i in range(beam_size):
        print('{} {}'.format(outs[i], scores[i]))

Streaming output truncated to the last 5000 lines.
Combined input shape: torch.Size([1, 128])
Combined input unsqueezed shape: torch.Size([1, 1, 128])
LSTM output shape: torch.Size([1, 1, 128])
Output shape after linear: torch.Size([1, 59])
Output shape after softmax: torch.Size([1, 59])
Input shape: torch.Size([1])
Car embeds shape: torch.Size([1, 110])
Cat embeds shape: torch.Size([1, 18])
Cat embeds reshaped shape: torch.Size([1, 18])
Combined input shape: torch.Size([1, 128])
Combined input unsqueezed shape: torch.Size([1, 1, 128])
LSTM output shape: torch.Size([1, 1, 128])
Output shape after linear: torch.Size([1, 59])
Output shape after softmax: torch.Size([1, 59])
Input shape: torch.Size([1])
Car embeds shape: torch.Size([1, 110])
Cat embeds shape: torch.Size([1, 18])
Cat embeds reshaped shape: torch.Size([1, 18])
Combined input shape: torch.Size([1, 128])
Combined input unsqueezed shape: torch.Size([1, 1, 128])
LSTM output shape: torch.Size([1, 1, 128])
Output shape after linea